In [1]:
#%%
from array import array
from cmath import nan
from pyexpat import model
import statistics
from tkinter.ttk import Separator
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchviz import make_dot
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torchvision import datasets, transforms
from torch.autograd import variable
from itertools import chain
from sklearn import metrics as met
import pickle
from icecream import ic

import matplotlib.pyplot as plt
import pathlib
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from importlib import reload
# import util
# import model_torch_simple
# from torchmetrics import Accuracy
from tqdm import tqdm
import argparse
from icecream import ic
import numpy as np
from PIL import Image
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(42)


/mnt/storageG1/lwang/miniconda3/envs/ml-g1/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def value_counts_list(lst):
    """
    Computes the frequency count of unique elements in a list and returns a dictionary, sorted by frequency count in
    descending order.

    Args:
    - lst (list): List of elements

    Returns:
    - dict: Dictionary with unique elements as keys and their frequency count as values, sorted by frequency count
    in descending order
    """
    value_counts = {}
    for item in lst:
        if item in value_counts:
            value_counts[item] += 1
        else:
            value_counts[item] = 1
    sorted_value_counts = dict(sorted(value_counts.items(), key=lambda x: x[1], reverse=True))
    return sorted_value_counts

def print_full(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

### Loading data

In [3]:
train_data = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/data/gene_seq_train.csv')
train_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/data/res_train.csv')
#don't touch test data, split out validation data from training data during training
test_data = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/data/gene_seq_test.csv')
test_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/data/res_test.csv')

In [4]:
N_samples = train_data.shape[0]
DRUGS = ['AMIKACIN',
 'CAPREOMYCIN',
 'CIPROFLOXACIN',
 'ETHAMBUTOL',
 'ETHIONAMIDE',
 'ISONIAZID',
 'KANAMYCIN',
 'LEVOFLOXACIN',
 'MOXIFLOXACIN',
 'OFLOXACIN',
 'PYRAZINAMIDE',
 'RIFAMPICIN',
 'STREPTOMYCIN']

DRUGS = train_target.columns
LOCI = train_data.columns
assert set(DRUGS) == set(train_target.columns)
N_drugs = len(DRUGS)

# Feature importance

In [5]:
def one_hot_torch(seq: str, dtype=torch.int8):
    seq_bytes = torch.ByteTensor(list(bytes(seq, "utf-8")))
    acgt_bytes = torch.ByteTensor(list(bytes("ACGT", "utf-8")))
    arr = torch.zeros(4, (len(seq_bytes)), dtype=dtype)
    arr[0, seq_bytes == acgt_bytes[0]] = 1
    arr[1, seq_bytes == acgt_bytes[1]] = 1
    arr[2, seq_bytes == acgt_bytes[2]] = 1
    arr[3, seq_bytes == acgt_bytes[3]] = 1
    return arr

# def one_hot_torch(seq):
#     oh = []
#     for sample in seq:
#         sample = torch.ByteTensor(list(bytes(sample, "utf-8")))
#         acgt_bytes = torch.ByteTensor(list(bytes("ACGT", "utf-8")))
#         arr = torch.zeros((len(sample), 4), dtype=torch.int8)
#         arr[sample == acgt_bytes[0], 0] = 1
#         arr[sample == acgt_bytes[1], 1] = 1
#         arr[sample == acgt_bytes[2], 2] = 1
#         arr[sample == acgt_bytes[3], 3] = 1
#         oh.append(arr)
#     return torch.stack(oh)

def my_padding(seq_tuple):
    list_x_ = list(seq_tuple)
    max_len = len(max(list_x_, key=len))
    for i, x in enumerate(list_x_):
        list_x_[i] = x + "N"*(max_len-len(x))
    return list_x_

#! faster than my_padding try to incorporate
def collate_padded_batch(batch):
    # get max length of seqs in batch
    max_len = max([x[0].shape[1] for x in batch])
    return torch.utils.data.default_collate(
        [(F.pad(x[0], (0, max_len - x[0].shape[1])), x[1]) for x in batch] #how does F.pad work
    )


def get_masked_loss(loss_fn):
    """
    Returns a loss function that ignores NaN values
    """

    def masked_loss(y_true, y_pred):
        y_pred = y_pred.view(-1, 13)  # Ensure y_pred has the same shape as y_true and non_nan_mask
        # ic(y_pred.shape)
        # ic(y_true.shape)
        non_nan_mask = ~y_true.isnan()
        # ic(non_nan_mask)
        y_true_non_nan = y_true[non_nan_mask]
        y_pred_non_nan = y_pred[non_nan_mask]

        return loss_fn(y_pred_non_nan, y_true_non_nan)

    return masked_loss

masked_MSE = get_masked_loss(torch.nn.MSELoss())


# Dateset

In [6]:
# Julian's code - implement this, might be faster
class OneHotSeqsDataset(torch.utils.data.Dataset): #? what's the difference between using inheritance and not?
    def __init__(
        self,
        seq_df,
        res_df,
        target_loci=LOCI,
        target_drugs=DRUGS,
        one_hot_dtype=torch.int8,
    ):
        self.seq_df = seq_df[target_loci]
        self.res_df = res_df[target_drugs]
        if not self.seq_df.index.equals(self.res_df.index):
            raise ValueError(
                "Indices of sequence and resistance dataframes don't match up"
            )
        self.one_hot_dtype = one_hot_dtype

    def __getitem__(self, index):
        """
        numerical index --> get `index`-th sample
        string index --> get sample with name `index`
        """
        if isinstance(index, int):
            seqs_comb = self.seq_df.iloc[index].str.cat()
            res = self.res_df.iloc[index]
        elif isinstance(index, str):
            seqs_comb = self.seq_df.loc[index].str.cat()
            res = self.res_df.loc[index]
        else:
            raise ValueError(
                "Index needs to be an integer or a sample name present in the dataset"
            )
        return one_hot_torch(seqs_comb, dtype=self.one_hot_dtype), torch.tensor(res)

    def __len__(self):
        return self.res_df.shape[0]
    
training_dataset = OneHotSeqsDataset(train_data, train_target, one_hot_dtype=torch.float)
train_dataset, val_dataset = random_split(training_dataset, [int(len(training_dataset)*0.8), len(training_dataset)-int(len(training_dataset)*0.8)])

# Model deepRam

In [7]:
class Network(nn.Module):
    def __init__(self, nummotif,motiflen,RNN_hidden_size,hidden_size,hidden,dropprob,sigmaConv,sigmaNeu,sigmaRNN,xavier_init):
      
        super(Network, self).__init__()
        
        self.hidden=hidden
        self.RNN_hidden_size=RNN_hidden_size
        
        self.dropprob=dropprob
        self.sigmaConv=sigmaConv
        self.sigmaNeu=sigmaNeu
        self.hidden_size=hidden_size  
        self.input_channels=4
        
        # Embedding
        if embedding:
              model1 = gensim.models.Word2Vec.load(word2vec_model)
              weights = torch.FloatTensor(model1.wv.vectors)
              self.embedding = nn.Embedding.from_pretrained(weights, freeze=False)
              self.input_channels=Embsize
              
        # Convnet
        self.ConvWeights=[]
        self.ConvBias=[]
        self.wConv=torch.randn(nummotif,self.input_channels,motiflen).to(device)
        self.wRect=torch.randn(nummotif).to(device)
        self.ConvWeights.append(self.wConv)
        self.ConvBias.append(self.wRect)
        conv_channels=nummotif
        
        if conv:
              self.FC_size= nummotif
              self.input_channels=nummotif
              for c in range(1,conv_layers):
                  Wconv=torch.randn(int(1.5*c*nummotif),conv_channels,motiflen).to(device)
                  Bconv=torch.randn(int(1.5*c*nummotif)).to(device)
                  self.ConvWeights.append(Wconv)
                  self.ConvBias.append(Bconv)
                  conv_channels=int(1.5*c*nummotif) # number of channels in the next layer
                  self.FC_size= int(1.5*c*nummotif)
                  self.input_channels=int(1.5*c*nummotif)
              torch.nn.init.normal_(self.ConvWeights[0],mean=0,std=sigmaConv)
              
              ind=0
              for weights in self.ConvWeights:
                  weights.requires_grad=True
                  if ind>0:
                    if dilation>1:
                      torch.nn.init.normal_(weights,mean=0,std=0.1)
                      print('ffffffff')
                    else:
                      torch.nn.init.xavier_uniform_(weights)
                  ind=ind+1
              for weights in self.ConvBias:
                  weights.requires_grad=True
                  torch.nn.init.normal_(weights)
            
        
        # RNN
        self.rnn = nn.GRU(self.input_channels, RNN_hidden_size, num_layers=1, bidirectional=False).to(device)
        if RNN:
              if RNN_type=='GRU':
                  self.rnn = nn.GRU(self.input_channels, RNN_hidden_size, num_layers=RNN_layers, bidirectional=False).to(device)
                  self.FC_size= RNN_hidden_size
              elif RNN_type=='BiGRU':
                  self.rnn = nn.GRU(self.input_channels, RNN_hidden_size, num_layers=RNN_layers, bidirectional=True).to(device)
                  self.FC_size= 2*RNN_hidden_size
              elif RNN_type=='LSTM':
                  self.rnn = nn.LSTM(self.input_channels, RNN_hidden_size, num_layers=RNN_layers, bidirectional=False).to(device)
                  self.FC_size= RNN_hidden_size
              elif RNN_type=='BiLSTM':
                  self.rnn = nn.LSTM(self.input_channels, RNN_hidden_size, num_layers=RNN_layers, bidirectional=True).to(device)
                  self.FC_size= 2*RNN_hidden_size
              if not xavier_init:
                for layer_p in self.rnn._all_weights:
                  for p in layer_p:
                    if 'weight' in p:
                      torch.nn.init.normal_(self.rnn.__getattr__(p),mean=0,std=sigmaRNN)
              else:
                for layer_p in self.rnn._all_weights:
                  for p in layer_p:
                    if 'weight' in p:
                      torch.nn.init.xavier_uniform_(self.rnn.__getattr__(p))

        # FC    
        self.wHidden=torch.randn(self.FC_size,self.hidden_size).to(device)
        self.wHiddenBias=torch.randn(self.hidden_size).to(device)
        self.wHidden.requires_grad=True
        self.wHiddenBias.requires_grad=True
        
        if not self.hidden:
            self.wNeu=torch.randn(self.FC_size,1).to(device)
            self.wNeuBias=torch.randn(1).to(device)
            if not xavier_init:
              torch.nn.init.normal_(self.wNeu,mean=0,std=self.sigmaNeu)
              torch.nn.init.normal_(self.wNeuBias,mean=0,std=self.sigmaNeu)
            else:
              torch.nn.init.xavier_uniform_(self.wNeu)
            
            

        else:
           
            self.wNeu=torch.randn(self.hidden_size,1).to(device)
            self.wNeuBias=torch.randn(1).to(device)     
            if not xavier_init:  
              torch.nn.init.normal_(self.wNeu,mean=0,std=self.sigmaNeu)
              torch.nn.init.normal_(self.wNeuBias,mean=0,std=self.sigmaNeu)
              torch.nn.init.normal_(self.wHidden,mean=0,std=self.sigmaNeu)
              torch.nn.init.normal_(self.wHiddenBias,mean=0,std=self.sigmaNeu)
            else:
              torch.nn.init.xavier_uniform(self.wNeu)
              torch.nn.init.xavier_uniform(self.wHidden)
            
  
           

        self.wNeu.requires_grad=True
        self.wNeuBias.requires_grad=True
        
        self.dropout=torch.nn.Dropout(p=dropprob, inplace=False)
        self.max=torch.nn.MaxPool1d(3, stride=1)
        
        
    def get_weights(self):
        ll=[]
        for layer_p in self.rnn._all_weights:
          for p in layer_p:
            if 'weight' in p:
               ll.append(self.rnn.__getattr__(p))
        return ll+self.ConvWeights+self.ConvBias
        
    def layer1out(self,x):
		
        if type(x) is np.ndarray:
          x = torch.from_numpy(x.astype(np.float32))
        #x = Variable(x, volatile=True)
        if torch.cuda.is_available():
          x = x.to(device)
        if embedding:
          print(x.shape)
          x= self.embedding(x)
          x=x.permute(0,2,1)
          print(x.shape)
        if conv:
          x=F.conv1d(x, self.wConv, bias=self.wRect, stride=1, padding=0)
          out=x.clamp(min=0)
          print(out.shape)
          temp = out.data.cpu().numpy()
        else:
          print('you need to have CNN to visualize motifs')
        return temp
        
    def forward(self, x):
      
        if embedding:
          # shape of x : batch_size x seq_len
          x= self.embedding(x)
          x=x.permute(0,2,1)
          # shape of x_emb : batch_size x embd_size x seq_len
          
#         else:
#           # shape of x : batch_size x 4 x seq_len
         
        if conv:
          x=F.conv1d(x, self.ConvWeights[0], bias=self.ConvBias[0], stride=1, padding=0)
          x=x.clamp(min=0)
          x=self.max(x)
          
          for c in range(1,len(self.ConvWeights)):
            x=F.conv1d(x, self.ConvWeights[c], bias=self.ConvBias[c], stride=1, padding=0,dilation=dilation)
            x=x.clamp(min=0)
            x=self.max(x)
            
        if RNN:
          if conv:
            
            x=self.dropout(x)
          x=x.permute(2,0,1)
          # shape of x :  seq_len x batch_size  x features
          output, _ = self.rnn(x)
          # shape of output :  seq_len x batch_size  x num_directions * features
          
          if RNN_type== 'BiLSTM' or RNN_type=='BiGRU':

              Normal_RNN=output[-1, :, :self.RNN_hidden_size]
              Rev_RNN=output[0, :, self.RNN_hidden_size:]
              x = torch.cat((Normal_RNN, Rev_RNN), 1)
              x=self.dropout(x)
              #shape of x: batch_size x 2*hidden_size
#               print(x.shape)
              
          else:
                      ## from (1, N, hidden) to (N, hidden)
              x = output[-1, :, :]
              x=self.dropout(x)
#               print(hn.shape)
#               x = hn.view(hn.size()[1], hn.size(2))
              # shape of x: batch_size x hidden_size
              #print(x.shape)
          
          
        else:
          x, _ = torch.max(x, dim=2)
          #print(x.shape)

          # shape of x : batch_size x numb_filters
          x=self.dropout(x)
        if self.hidden:
          x=x @ self.wHidden
          x.add_(self.wHiddenBias)
          x=x.clamp(min=0)
          x=self.dropout(x)
        x=x @ self.wNeu
        x.add_(self.wNeuBias)
          
        
        
        return torch.sigmoid(x)

In [8]:

torch.cuda.empty_cache()

embedding=False
conv=True
RNN=True
RNN_type='BiLSTM'

bases='ACGT' #DNA bases
basesRNA='ACGU'#RNA bases
batch_size=128
evaluate_performance=False
train=True
model_dir='models/'
# embedding hyper-parameters
Embepochs=100 #number of epochs to train word2vec
Embsize=50 # 
kmer_len=3
stride=1
word2vect_train=True
word2vec_model='models/word2vec_model'
# CNN hyper-parameters
nummotif=16 #number of motifs to discover
motiflen=24
sigmaNeu=10**-6
sigmaConv = 10**-6
sigmaNeu = 10**-3
sigmaRNN = 10**-4
xavier_init=True
RNN_hidden_size = 64
hidden_size = 64 
hidden = False
dropprob = 0.2

######### Global variables #########

global embedding
global conv
global RNN
global RNN_type
global kmer_len
global stride
global word2vec_train 
global word2vec_model 
global evaluate_performance
global train
global model_dir
global model_path
global out_file
global motif
global motif_dir
global tomtom_dir
global data_type
global conv_layers
global RNN_layers
global dilation

# Default values for global variables
embedding = False
conv = True
RNN = True
RNN_type = 'GRU'  # Common types are 'GRU', 'LSTM', 'BiGRU', 'BiLSTM'
kmer_len = 3  # Common k-mer lengths are 3, 4, or 5
stride = 1
word2vec_train = False
word2vec_model = 'word2vec.model'  # Default path to a Word2Vec model file
evaluate_performance = False
train = True
model_dir = 'models/'  # Default directory to save models
model_path = 'models/model.pth'  # Default path to save a specific model
out_file = 'output.txt'  # Default output file name
motif = False
motif_dir = 'motifs/'  # Default directory for motifs
tomtom_dir = 'tomtom/'  # Default directory for tomtom tool outputs
data_type = 'sequence'  # Common types might be 'sequence', 'image', 'text', etc.
conv_layers = 2  # Default number of convolutional layers
RNN_layers = 1  # Default number of RNN layers
dilation = 1  # Default dilation value in convolutional layers

deepRam = Network(nummotif,motiflen,RNN_hidden_size,hidden_size,hidden,dropprob,sigmaConv,sigmaNeu,sigmaRNN,xavier_init)

In [9]:
def Calibration():
	print('start')
	best_AUC=0
	device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
	print(device)
	# device='cpu'
	learning_steps_list=[5000,10000,15000,20000,25000,30000,35000,40000]
	for number in range(40):
		# hyper-parameters
		RNN_hidden_size_list=[20,50,80,100]
		RNN_hidden_size=random.choice(RNN_hidden_size_list)
		dropoutList=[0,0.15,0.3,0.45,0.6] 
		dropprob=random.choice(dropoutList)
		hidden_list=[True,False]
		hidden=random.choice(hidden_list)
		xavier_List=[True,True,False] 
		xavier=random.choice(xavier_List)
		hidden_size_list=[32,64]
		hidden_size=random.choice(hidden_size_list)
		optim_list=['SGD','Adagrad','Adagrad']
		optim=random.choice(optim_list)
		learning_rate=logsampler(0.005,0.5) 
		momentum_rate=sqrtsampler(0.95,0.99)  
		sigmaConv=logsampler(10**-6,10**-2)   
		sigmaNeu=logsampler(10**-3,10**-1) 
		sigmaRNN=logsampler(10**-4,10**-1) 
		weightDecay=logsampler(10**-10,10**-1) 
		nummotif_list=[16]
		nummotif1=random.choice(nummotif_list)
		
		
		
	
		
		model_auc=[[],[],[]]
		for kk in range(3):
			model = Network(nummotif1,motiflen,RNN_hidden_size,hidden_size,hidden,dropprob,sigmaConv,sigmaNeu,sigmaRNN,xavier).to(device)
			if optim=='SGD':
				optimizer = torch.optim.SGD(model.get_weights()+[model.wNeu,model.wNeuBias,model.wHidden,model.wHiddenBias], lr=learning_rate,momentum=momentum_rate,nesterov=True
											,weight_decay=weightDecay)
			else:
				optimizer = torch.optim.Adagrad(model.get_weights()+[model.wNeu,model.wNeuBias,model.wHidden,model.wHiddenBias], lr=learning_rate,weight_decay=weightDecay)

			train_loader=train_dataloader[kk]
			valid_loader=valid_dataloader[kk]
			
			
			learning_steps=0
			while learning_steps<=40000:
			   
				auc=[]
				model.train()
				for i, (data, target) in enumerate(train_loader):
					
					data = data.to(device)
					target = target.to(device)

					# Forward pass
					output = model(data)          
					loss = F.binary_cross_entropy(output,target)
					optimizer.zero_grad()
					loss.backward()
					optimizer.step()
					learning_steps+=1

					if learning_steps% 5000==0:
						
							
							
							
						with torch.no_grad():
							model.eval()
							auc=[]
							for j, (data1, target1) in enumerate(valid_loader):
								data1 = data1.to(device)
								target1 = target1.to(device)
								
								# Forward pass
								output = model(data1)
								
								pred=output.cpu().detach().numpy().reshape(output.shape[0])
								labels=target1.cpu().numpy().reshape(output.shape[0])
								if output.shape[0]>60:
									auc.append(metrics.roc_auc_score(labels, pred))
							#print(np.mean(auc))
							model_auc[kk].append(np.mean(auc))
							
							model.train()
						
		
		print('                   ##########################################               ')

		for n in range(8):
			AUC=(model_auc[0][n]+model_auc[1][n]+model_auc[2][n])/3
			#print(AUC)
			if AUC>best_AUC:
				best_AUC=AUC
				best_learning_steps=learning_steps_list[n]
				best_LearningRate=learning_rate
				best_LearningMomentum=momentum_rate
				best_sigmaConv=sigmaConv
				best_dropprob=dropprob
				best_sigmaNeu=sigmaNeu
				best_RNN_hidden_size=RNN_hidden_size
				best_weightDecay=weightDecay
				best_hidden=hidden
				best_sigmaRNN=sigmaRNN
				best_xavier=xavier
				best_optim=optim
				best_nummotif=nummotif
				best_hidden_size=hidden_size


	print('best_AUC=',best_AUC)            
	print('best_learning_steps=',best_learning_steps)      
	print('best_LearningRate=',best_LearningRate)
	print('best_LearningMomentum=',best_LearningMomentum)
	print('best_sigmaConv=',best_sigmaConv)
	print('best_dropprob=',best_dropprob)
	print('best_sigmaNeu=',best_sigmaNeu)
	print('best_RNN_hidden_size',best_RNN_hidden_size)
	print('best_weightDecay=',weightDecay)
	print('best_hidden=',best_hidden)
	print('best_sigmaRNN=',best_sigmaRNN)
	print('best_xavier=',best_xavier)
	print('best_optim=',best_optim)
	print('best_nummotif=',best_nummotif)
	print('best_hidden_size=',best_hidden_size)
	
	best_hyperparameters = {'best_learning_steps': best_learning_steps,'best_LearningRate':best_LearningRate,'best_LearningMomentum':best_LearningMomentum,'best_sigmaConv':best_sigmaConv,
	                         'best_dropprob':best_dropprob,'best_sigmaNeu':best_sigmaNeu,'best_RNN_hidden_size':best_RNN_hidden_size,
	                         'best_weightDecay':best_weightDecay,'best_hidden':best_hidden,'best_sigmaRNN':best_sigmaRNN,'best_xavier':best_xavier,'best_optim':best_optim,'best_nummotif':best_nummotif,'best_hidden_size':best_hidden_size}
	torch.save(best_hyperparameters, model_dir+'best_hyperpamarameters.pth')
	return best_hyperparameters

# Training deepram

In [11]:
ic.disable()
# ic.enable()
train_epoch_loss = []
test_epoch_loss = []

epoch = 10

for e in tqdm(range(1, epoch+1)):
    model.train()
    train_batch_loss = []
    test_batch_loss = []
    
    for x, y in train_loader:
        x_batch = torch.squeeze(x, 0).to(device)
        y_batch = y.to(device)
        x_batch = x_batch.float()
        y_batch = y_batch.float()
        # y_batch = y_batch.view(-1)

        # y_batch = one_hot_torch(y).to(device)
        # print('batch y size before flatten:',y_batch.size())
        # y_batch = y_batch.flatten()
        # print('batch y size after flatten:',y_batch.size())
        # print(x_batch.size())
        # print(x_batch.size())
# For example, if you have a convolutional layer with 64 output channels, 3 input channels, and a kernel size of 3x3, the weight parameters would have a dimension of (64, 3, 3, 3)
        # print(x_batch.size())
        pred = deepRam(x_batch.float())
        # print(x_batch)
        # print(pred)
        # pred = pred.unsqueeze(0)
        # ic(pred)
        # ic(y_batch)
        loss_train = criterion(y_batch, pred)
        ic(loss_train)
        train_batch_loss.append(loss_train)
        
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        # print(f'Batch - GPU Memory Allocated: {torch.cuda.memory_allocated() / 1024**2:.2f} MB')

    train_epoch_loss.append(torch.mean(torch.stack(train_batch_loss)).detach().cpu().numpy())
    with torch.no_grad():
        # print('test')
        for x, y in test_loader:
            x_batch = x.to(device)
            y_batch = y.to(device)
            # print(x_batch.size())
            # y_batch = torch.Tensor.float(y).to(device)
            # x_batch = x_batch.permute(0, 3, 1, 2).to(device)
            pred = model(x_batch.float())
            # pred = pred.unsqueeze(0)

            loss_test = criterion(y_batch, pred)
            test_batch_loss.append(loss_test)
        test_epoch_loss.append(torch.mean(torch.stack(test_batch_loss)).detach().cpu().numpy())
    print(f'Epoch {e}')
    print(f"Training loss: {torch.mean(torch.stack(train_batch_loss)).detach().cpu().numpy()}")
    print(f"Validation loss: {torch.mean(torch.stack(test_batch_loss)).detach().cpu().numpy()}") 
    # print(train_batch_loss)
    # print(test_batch_loss)
    # print(f"Training loss: {np.mean(train_batch_loss)}")
    # print(f"Validation loss: {np.mean(test_batch_loss)}")
    print('==='*10)

  0%|          | 0/10 [00:00<?, ?it/s]


AttributeError: module 'pyexpat.model' has no attribute 'train'